### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import pandas as pd

# File to Load (Remember to Change These)
school_data_to_load = "Resources/schools_complete.csv"
student_data_to_load = "Resources/students_complete.csv"

# Read School and Student Data File and store into Pandas Data Frames
school_data = pd.read_csv(school_data_to_load)
student_data = pd.read_csv(student_data_to_load)

# Combine the data into a single dataset - I changed the order of the data sets and made it a right join instead of left.
# That way it's a little easier for me to deal with later on. 
school_data_complete = pd.merge(school_data, student_data, how="right", on=["school_name", "school_name"])
school_data_complete = school_data_complete.sort_values(["school_name"])
school_data_complete.head()

,School ID,school_name,type,size,budget,Student ID,student_name,gender,grade,reading_score,math_score
19584,7,Bailey High School,District,4976,3124928,19584,Tammie Fox,F,11th,82,92
21193,7,Bailey High School,District,4976,3124928,21193,Jennifer Murray,F,9th,88,89
21192,7,Bailey High School,District,4976,3124928,21192,Lisa Pineda,F,9th,86,67
21191,7,Bailey High School,District,4976,3124928,21191,Cameron Miller,M,11th,70,75
21190,7,Bailey High School,District,4976,3124928,21190,Thomas Rasmussen,M,12th,77,82


## District Summary

* Calculate the total number of schools

* Calculate the total number of students

* Calculate the total budget

* Calculate the average math score 

* Calculate the average reading score

* Calculate the overall passing rate (overall average score), i.e. (avg. math score + avg. reading score)/2

* Calculate the percentage of students with a passing math score (70 or greater)

* Calculate the percentage of students with a passing reading score (70 or greater)

* Create a dataframe to hold the above results

* Optional: give the displayed data cleaner formatting

In [2]:
#Calculate the total number of schools
# The value_counts method counts unique values in a column
school_count = school_data["school_name"].count()
student_count = student_data["student_name"].count()
total_budget = school_data["budget"].sum()
avg_read = student_data["reading_score"].mean()
avg_math = student_data["math_score"].mean()
overall_pass = (avg_read + avg_math)/2

#math
passed_math = [score for score in student_data["math_score"] if score >= 70]
pass_math = len(passed_math)
perc_pass_math = (pass_math/student_count) * 100
#perc_pass_math

#reading
passed_reading = [score for score in student_data["reading_score"] if score >= 70]
pass_reading = len(passed_reading)
perc_pass_read = (pass_reading/student_count) * 100
#perc_pass_read


district_summary_df = pd.DataFrame({"Total Schools": [school_count],
                                    "Total Students": [student_count],
                                    "Total Budget": [total_budget],
                                    "Average Math Score": [avg_math],
                                    "Average Reading Score": [avg_read],
                                    "% Passing Math": [perc_pass_math],
                                    "% Passing Reading": [perc_pass_read],
                                    "% Overall Passing Rate": [overall_pass]
                                   })

district_summary_df = district_summary_df.round(6)

district_summary_df["% Passing Math"] = district_summary_df["% Passing Math"].map("{0:,.2f}%".format)
district_summary_df["% Passing Reading"] = district_summary_df["% Passing Reading"].map("{0:,.2f}%".format)
district_summary_df["% Overall Passing Rate"] = district_summary_df["% Overall Passing Rate"].map("{0:,.2f}%".format)
district_summary_df["Total Budget"] = district_summary_df["Total Budget"].map("${0:,.0f}".format)

district_summary_df

,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
0,15,39170,"$24,649,428",78.985371,81.87784,74.98%,85.81%,80.43%


## School Summary

* Create an overview table that summarizes key metrics about each school, including:
  * School Name
  * School Type
  * Total Students
  * Total School Budget
  * Per Student Budget
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)
  
* Create a dataframe to hold the above results

In [13]:
#Create data frame containing all necessary data for school summary questions
#Using "school_name", "type", "size", "budget", "gender", "reading_score", "math_score"
#school_data_dist = pd.DataFrame(school_data_complete[["school_name", "type", "size", "budget", "gender", "reading_score", "math_score"]])


#create a data frame grouped by schools to determine list of schools, enrollment per school, total budget per school. 
#grouped_schools_df = school_data_dist.groupby(["school_name"])


school_name = school_data_complete["school_name"].unique()
enrollment = school_data_complete["size"].unique()
total_budget_school = school_data_complete["budget"].unique()
budget_per_stu = total_budget_school/enrollment


school_summary_df = pd.DataFrame({"school_name":school_name,
                                  "Enrollment": enrollment,
                                  "Total Budget": total_budget_school,
                                  "Budget per Student":budget_per_stu })

#school_summary_df.groupby(["school_name"])

grouped_schools_df = school_data_complete.groupby(["school_name"])
 

#calculate the average math and reading scores per school
avg_math_school = grouped_schools_df["math_score"].mean()
avg_read_school = grouped_schools_df["reading_score"].mean()

#calculate number of students passing reading and math.
grouped_schools_math_df = school_data_complete.loc[school_data_complete["math_score"] >= 70].groupby(["school_name"])
grouped_schools_read_df = school_data_complete.loc[school_data_complete["reading_score"] >= 70].groupby(["school_name"])

pass_math = grouped_schools_math_df["math_score"].count()
pass_reading = grouped_schools_read_df["reading_score"].count()

#calculate percentage of students passing reading and math
perc_passing_math = (pass_math/enrollment)*100 
perc_passing_reading = (pass_reading/enrollment)*100 

#Overall passing rate per school
overall_pass_school = (perc_passing_reading + perc_passing_math)/2


school_avg_df = pd.DataFrame({"Average Math": avg_math_school,
                             "Average Reading": avg_read_school })
                              

school_perc_df = pd.DataFrame({"% Passing Math": perc_passing_math,
                               #"# passing math": pass_math,
                              "% Passing Reading": perc_passing_reading,
                               #"# passing reading": pass_reading,
                              "% Passing Overall": overall_pass_school})
                              

#Format data in dataframe

school_dist_summary = pd.merge(school_summary_df, school_avg_df, how="left", on=["school_name", "school_name"])
school_dist_summ_perc = pd.merge(school_dist_summary, school_perc_df, how="right", on=["school_name", "school_name"])

school_dist_summ_perc["% Passing Math"] = school_dist_summ_perc["% Passing Math"].map("{0:,.2f}%".format)
school_dist_summ_perc["% Passing Reading"] = school_dist_summ_perc["% Passing Reading"].map("{0:,.2f}%".format)
school_dist_summ_perc["% Passing Overall"] = school_dist_summ_perc["% Passing Overall"].map("{0:,.2f}%".format)
school_dist_summ_perc["Total Budget"] = school_dist_summ_perc["Total Budget"].map("${0:,.0f}".format)
school_dist_summ_perc["Budget per Student"] = school_dist_summ_perc["Budget per Student"].map("${0:,.0f}".format)

school_dist_summ_perc

#school_summary_df
#school_avg_df
#school_dist_summary
#print(f'passed math: {pass_math} Enrollment: {enrollment} Total budget: {total_budget_school}')                            

,school_name,Enrollment,Total Budget,Budget per Student,Average Math,Average Reading,% Passing Math,% Passing Reading,% Passing Overall
0,Bailey High School,4976,"$3,124,928",$628,77.048432,81.033963,66.68%,81.93%,74.31%
1,Cabrera High School,1858,"$1,081,356",$582,83.061895,83.975780,94.13%,97.04%,95.59%
2,Figueroa High School,2949,"$1,884,411",$639,76.711767,81.158020,65.99%,80.74%,73.36%
3,Ford High School,2739,"$1,763,916",$644,77.102592,80.746258,68.31%,79.30%,73.80%
4,Griffin High School,1468,"$917,500",$625,83.351499,83.816757,93.39%,97.14%,95.27%
5,Hernandez High School,4635,"$3,022,020",$652,77.289752,80.934412,66.75%,80.86%,73.81%
6,Holden High School,427,"$248,087",$581,83.803279,83.814988,92.51%,96.25%,94.38%
7,Huang High School,2917,"$1,910,635",$655,76.629414,81.182722,65.68%,81.32%,73.50%
8,Johnson High School,4761,"$3,094,650",$650,77.072464,80.966394,66.06%,81.22%,73.64%
9,Pena High School,962,"$585,858",$609,83.839917,84.044699,94.59%,95.95%,95.27%


## Top Performing Schools (By Passing Rate)

* Sort and display the top five schools in overall passing rate

In [14]:
# Display top 5 performing schools in descending order
top_school_pass_df = school_dist_summ_perc.sort_values(["% Passing Overall"], ascending=False)
top_school_pass_df.head()





,school_name,Enrollment,Total Budget,Budget per Student,Average Math,Average Reading,% Passing Math,% Passing Reading,% Passing Overall
1,Cabrera High School,1858,"$1,081,356",$582,83.061895,83.975780,94.13%,97.04%,95.59%
12,Thomas High School,1635,"$1,043,130",$638,83.418349,83.848930,93.27%,97.31%,95.29%
4,Griffin High School,1468,"$917,500",$625,83.351499,83.816757,93.39%,97.14%,95.27%
9,Pena High School,962,"$585,858",$609,83.839917,84.044699,94.59%,95.95%,95.27%
13,Wilson High School,2283,"$1,319,574",$578,83.274201,83.989488,93.87%,96.54%,95.20%


## Bottom Performing Schools (By Passing Rate)

* Sort and display the five worst-performing schools

In [15]:
# Display bottom 5 performing schools in ascending order
top_school_pass_df = school_dist_summ_perc.sort_values(["% Passing Overall"], ascending=True)
top_school_pass_df.head()

,school_name,Enrollment,Total Budget,Budget per Student,Average Math,Average Reading,% Passing Math,% Passing Reading,% Passing Overall
10,Rodriguez High School,3999,"$2,547,363",$637,76.842711,80.744686,66.37%,80.22%,73.29%
2,Figueroa High School,2949,"$1,884,411",$639,76.711767,81.158020,65.99%,80.74%,73.36%
7,Huang High School,2917,"$1,910,635",$655,76.629414,81.182722,65.68%,81.32%,73.50%
8,Johnson High School,4761,"$3,094,650",$650,77.072464,80.966394,66.06%,81.22%,73.64%
3,Ford High School,2739,"$1,763,916",$644,77.102592,80.746258,68.31%,79.30%,73.80%


## Math Scores by Grade

* Create a table that lists the average Reading Score for students of each grade level (9th, 10th, 11th, 12th) at each school.

  * Create a pandas series for each grade. Hint: use a conditional statement.
  
  * Group each series by school
  
  * Combine the series into a dataframe
  
  * Optional: give the displayed data cleaner formatting

In [16]:
school_data_complete


        

,School ID,school_name,type,size,budget,Student ID,student_name,gender,grade,reading_score,math_score
19584,7,Bailey High School,District,4976,3124928,19584,Tammie Fox,F,11th,82,92
21193,7,Bailey High School,District,4976,3124928,21193,Jennifer Murray,F,9th,88,89
21192,7,Bailey High School,District,4976,3124928,21192,Lisa Pineda,F,9th,86,67
21191,7,Bailey High School,District,4976,3124928,21191,Cameron Miller,M,11th,70,75
21190,7,Bailey High School,District,4976,3124928,21190,Thomas Rasmussen,M,12th,77,82
...,...,...,...,...,...,...,...,...,...,...,...
24829,10,Wright High School,Charter,1800,1049400,24829,John Lozano,M,11th,84,77
24828,10,Wright High School,Charter,1800,1049400,24828,Jonathan Thomas,M,12th,95,80
24827,10,Wright High School,Charter,1800,1049400,24827,Lori Ramirez,F,10th,74,74
24797,10,Wright High School,Charter,1800,1049400,24797,Scott Scott,M,10th,90,83


## Reading Score by Grade 

* Perform the same operations as above for reading scores

## Scores by School Spending

* Create a table that breaks down school performances based on average Spending Ranges (Per Student). Use 4 reasonable bins to group school spending. Include in the table each of the following:
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)

In [6]:
# Sample bins. Feel free to create your own bins.
spending_bins = [0, 585, 615, 645, 675]
group_names = ["<$585", "$585-615", "$615-645", "$645-675"]

## Scores by School Size

* Perform the same operations as above, based on school size.

In [7]:
# Sample bins. Feel free to create your own bins.
size_bins = [0, 1000, 2000, 5000]
group_names = ["Small (<1000)", "Medium (1000-2000)", "Large (2000-5000)"]

## Scores by School Type

* Perform the same operations as above, based on school type.